# Quantile Regression

Estimate conditional quantiles (e.g., median and P90) of revenue to understand upside/downside risk.

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, warnings
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import QuantileRegressor
from sklearn.metrics import mean_pinball_loss

!wget -q https://raw.githubusercontent.com/Jihun-ust/ust-mail-557/main/Regression_Forecasting/reg_for_utils.py
import reg_for_utils as utils
csv_path = "https://raw.githubusercontent.com/Jihun-ust/ust-mail-557/main/Regression_Forecasting/marketing_daily.csv"
warnings.filterwarnings("ignore")

df = pd.read_csv(csv_path, parse_dates=["date"]).sort_values("date")
train, test = utils.time_train_test_split(df, "date", test_days=90)

X_cols = ["search_spend","social_spend","display_spend","promo","price_index","temp_F","rain","is_weekend"]
y_col = "revenue"

pre = ColumnTransformer([("num", StandardScaler(), ["search_spend","social_spend","display_spend","price_index","temp_F"]),
                         ("cat", OneHotEncoder(drop="if_binary"), ["promo","rain","is_weekend"])])

taus = [0.5, 0.9]
models = {}
for t in taus:
    qr = Pipeline([("pre", pre), ("est", QuantileRegressor(quantile=t, alpha=0.0001, solver="highs"))])
    qr.fit(train[X_cols], train[y_col])
    models[t] = qr
    pred = qr.predict(test[X_cols])
    loss = mean_pinball_loss(test[y_col], pred, alpha=t)
    print("tau=", t, " pinball_loss=", round(loss,3), " MAE=", round(utils.mae(test[y_col], pred),2))

med = models[0.5].predict(test[X_cols])
p90 = models[0.9].predict(test[X_cols])

plt.figure(figsize=(10,4))
plt.plot(test["date"], test[y_col], label="actual")
plt.plot(test["date"], med, label="median")
plt.plot(test["date"], p90, label="p90")
plt.title("Revenue: median and P90 quantile regression"); plt.legend(); plt.tight_layout(); plt.show()

### Evaluation Examples
- Set up predictions & helpers

In [ ]:
# Collect predictions & helpers
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.metrics import mean_pinball_loss

X_cols = X_cols
y_col  = y_col

# Ensure we have predictions for taus in the 'models' dict
preds = {t: models[t].predict(test[X_cols]) for t in models.keys()}
y_true = test[y_col].to_numpy()

# Combine for analysis
eval_df = test[[c for c in test.columns if c in (["date"] + X_cols + [y_col])]].copy()
for t, yhat in preds.items():
    eval_df[f"q{int(t*100)}"] = yhat
eval_df["y_true"] = y_true

def empirical_coverage(y, qhat):
    """Fraction of y <= predicted quantile (should be ~ tau)."""
    y = np.asarray(y); qhat = np.asarray(qhat)
    return float(np.mean(y <= qhat))

def sharpness_width(q_hi, q_lo):
    """Average interval width (lower is 'sharper')."""
    return float(np.mean(np.maximum(0.0, q_hi - q_lo)))

def scale_normalizer(y):
    """Robust scale for normalized sharpness."""
    return np.subtract(*np.percentile(y, [90, 10]))  # P90 - P10

print("Rows in evaluation:", len(eval_df))
eval_df.head(3)

#### Pinball loss (tabular summary)

In [ ]:
# Pinball loss summary across taus
rows = []
for tau in sorted(models.keys()):
    yhat = eval_df[f"q{int(tau*100)}"].to_numpy()
    rows.append({
        "tau": tau,
        "pinball_loss": round(mean_pinball_loss(y_true, yhat, alpha=tau), 4),
        "MAE": round(np.mean(np.abs(y_true - yhat)), 3)
    })
pinball_table = pd.DataFrame(rows).sort_values("tau").reset_index(drop=True)
pinball_table

#### Empirical coverage (one‑sided for each tau)
- Target is ≈τ. Large deviations imply miscalibration in certain regions.

In [ ]:
# Empirical coverage: P(y <= q_tau(x)) should ≈ tau
cov_rows = []
for tau in sorted(models.keys()):
    qcol = f"q{int(tau*100)}"
    cov = empirical_coverage(eval_df["y_true"], eval_df[qcol])
    cov_rows.append({"tau": tau, "empirical_coverage": round(cov, 3)})
cov_df = pd.DataFrame(cov_rows)
display(cov_df)

# Quick bar chart
plt.figure(figsize=(6,4))
plt.bar([str(r["tau"]) for r in cov_rows], [r["empirical_coverage"] for r in cov_rows])
for r in cov_rows:
    plt.axhline(r["tau"], linestyle="--", linewidth=1)
plt.ylim(0, 1)
plt.ylabel("Empirical coverage  (P[y ≤ q̂τ])")
plt.xlabel("τ")
plt.title("Calibration — Empirical coverage vs target τ")
plt.tight_layout(); plt.show()

#### Interval sharpness (requires both median & P90; normalizes too)
- Smaller average width is better (trade‑off with coverage); a normalized version lets you compare across datasets.

In [ ]:
# Interval sharpness between median (P50) and P90
needs = {"q50", "q90"}
if needs.issubset(set(eval_df.columns)):
    width = eval_df["q90"] - eval_df["q50"]
    avg_width = sharpness_width(eval_df["q90"], eval_df["q50"])
    norm = scale_normalizer(eval_df["y_true"])
    norm_width = avg_width / (norm if norm > 0 else 1.0)

    print(f"Average width (P50→P90): {avg_width:.3f}")
    print(f"Normalized width (÷(P90−P10) of y): {norm_width:.3f}")

    # Plot width vs a key driver (choose one you care about)
    key = "search_spend" if "search_spend" in eval_df.columns else X_cols[0]
    plt.figure(figsize=(7,4))
    plt.scatter(eval_df[key], width, alpha=0.35, s=14)
    plt.xlabel(key); plt.ylabel("Interval width (q90 − q50)")
    plt.title("Interval sharpness across feature values")
    plt.tight_layout(); plt.show()
else:
    print("Sharpness: need both q50 and q90 predictions; skipping.")

#### Calibration curves (bin by predicted quantile)
- Show how often actuals fall below predicted quantiles across the range of predictions (reference = horizontal line at τ).

In [ ]:
# Calibration curves per τ: bin by predicted q̂τ and check empirical P(y ≤ q̂τ)
def calibration_curve(y, qhat, n_bins=10):
    order = np.argsort(qhat)
    y_sorted = y[order]
    q_sorted = qhat[order]
    bins = np.array_split(np.arange(len(y_sorted)), n_bins)
    frac = []
    q_mid = []
    for b in bins:
        frac.append(np.mean(y_sorted[b] <= q_sorted[b]))
        q_mid.append(np.median(q_sorted[b]))
    return np.array(q_mid), np.array(frac)

plt.figure(figsize=(6,5))
for tau in sorted(models.keys()):
    q = eval_df[f"q{int(tau*100)}"].to_numpy()
    q_mid, frac = calibration_curve(eval_df["y_true"].to_numpy(), q, n_bins=10)
    plt.plot(q_mid, frac, marker="o", label=f"τ={tau:.2f}")
# Ideal reference lines: horizontal at τ (not a straight diagonal!)
for tau in sorted(models.keys()):
    plt.axhline(tau, linestyle="--", linewidth=1)
plt.xlabel("Predicted quantile value q̂τ")
plt.ylabel("Empirical P(y ≤ q̂τ)")
plt.title("Calibration curves by predicted quantile value")
plt.legend()
plt.tight_layout(); plt.show()

#### Rolling validation (time‑aware pinball, coverage, sharpness)
- See stability/drift over time for pinball, coverage, and interval width. Adjust window to your cadence (e.g., 12 weeks).

In [ ]:
# Rolling validation over time (if a date column exists)
date_col = "date" if "date" in eval_df.columns else None
if date_col is None:
    # Try to infer a datetime index
    if np.issubdtype(eval_df.index.dtype, np.datetime64):
        eval_df = eval_df.reset_index().rename(columns={"index": "date"})
        date_col = "date"

if date_col is None:
    print("No 'date' column or datetime index found; skipping rolling validation.")
else:
    # Ensure datetime & sort
    eval_df[date_col] = pd.to_datetime(eval_df[date_col], errors="coerce")
    ev = eval_df.dropna(subset=[date_col]).sort_values(date_col).copy()

    window = 56  # ~8 weeks if daily; adjust to your cadence
    metrics = []
    for start in range(0, len(ev) - window + 1):
        sub = ev.iloc[start:start+window]
        row = {
            "end_date": sub[date_col].iloc[-1],
        }
        # Pinball & coverage for each tau
        for tau in sorted(models.keys()):
            qcol = f"q{int(tau*100)}"
            row[f"pinball_tau{int(tau*100)}"] = mean_pinball_loss(sub["y_true"], sub[qcol], alpha=tau)
            row[f"cover_tau{int(tau*100)}"]   = empirical_coverage(sub["y_true"], sub[qcol])
        # Sharpness (if both available)
        if {"q50","q90"}.issubset(set(sub.columns)):
            row["sharp_width_p50_p90"] = sharpness_width(sub["q90"], sub["q50"])
        metrics.append(row)

    roll = pd.DataFrame(metrics)

    # Plot rolling pinball for τ=0.5 and τ=0.9 (if present)
    plt.figure(figsize=(9,4))
    if "pinball_tau50" in roll.columns:
        plt.plot(roll["end_date"], roll["pinball_tau50"], label="Pinball τ=0.5")
    if "pinball_tau90" in roll.columns:
        plt.plot(roll["end_date"], roll["pinball_tau90"], label="Pinball τ=0.9")
    plt.title("Rolling Pinball Loss"); plt.xlabel("End date"); plt.ylabel("Loss")
    plt.legend(); plt.tight_layout(); plt.show()

    # Rolling coverage
    plt.figure(figsize=(9,4))
    if "cover_tau50" in roll.columns:
        plt.plot(roll["end_date"], roll["cover_tau50"], label="Coverage τ=0.5")
        plt.axhline(0.5, linestyle="--", linewidth=1)
    if "cover_tau90" in roll.columns:
        plt.plot(roll["end_date"], roll["cover_tau90"], label="Coverage τ=0.9")
        plt.axhline(0.9, linestyle="--", linewidth=1)
    plt.title("Rolling Empirical Coverage"); plt.xlabel("End date"); plt.ylabel("P(y ≤ q̂τ)")
    plt.legend(); plt.tight_layout(); plt.show()

    # Rolling sharpness
    if "sharp_width_p50_p90" in roll.columns:
        plt.figure(figsize=(9,4))
        plt.plot(roll["end_date"], roll["sharp_width_p50_p90"], label="Width q90−q50")
        plt.title("Rolling Interval Sharpness (q90−q50)"); plt.xlabel("End date"); plt.ylabel("Width")
        plt.tight_layout(); plt.show()

    roll.tail(10)